<a href="https://colab.research.google.com/github/MorgooN/ML-Labs/blob/main/LAB2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import keras
from keras.layers import Rescaling, Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from keras import Model, Input
from keras.callbacks import ModelCheckpoint

In [2]:
batch_size = 32
img_height = 224
img_width = 224

In [8]:
!gdown --id 1qtwiBTaR5jE_vDEUW_HA4nXGLPTLCyBl # downloading saved dataset from google drive

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1qtwiBTaR5jE_vDEUW_HA4nXGLPTLCyBl
To: /content/Dataset.zip
100% 849M/849M [00:16<00:00, 51.2MB/s]


In [ ]:
!unzip Dataset.zip # unpack zip fils

In [11]:
train_ds = keras.utils.image_dataset_from_directory(
  directory= "/content/Farm Harmful Animal Dataset/test", # path to test data
  label_mode = 'int',       # correct answers as numbers(Label Encoding): 0, 1, 2, 3 etc.
  color_mode='rgb',         # each image as RGB
  batch_size=batch_size,    # batch-size regulator
  seed=123,
  image_size=(img_height, img_width),
  shuffle=True)

Found 899 files belonging to 15 classes.


In [12]:
val_ds = keras.utils.image_dataset_from_directory(
  directory= "/content/Farm Harmful Animal Dataset/validation",  # path to val. data
  label_mode = 'int',
  color_mode='rgb',
  batch_size=batch_size,
  seed=123,
  image_size=(img_height, img_width),
  shuffle=False) # disable shuffle for a val. data

Found 923 files belonging to 15 classes.


In [13]:
test_ds = keras.utils.image_dataset_from_directory(
  directory= "/content/Farm Harmful Animal Dataset/test",  # path to test. data
  label_mode = 'int',
  color_mode='rgb',
  batch_size=batch_size,
  seed=123,
  image_size=(img_height, img_width),
  shuffle=False) # also disable shuffle for a test. data

Found 899 files belonging to 15 classes.
